In [93]:
import pyscf
import scipy
import numpy as np
from pyscf.pbc import gto, scf


In [2]:

cell = gto.Cell()
cell.atom = '''
H  0 0 0
H  .75 0 0
'''
cell.basis = 'gth-dzvp'
cell.pseudo = 'gth-pade'
cell.a = np.eye(3) * 2

cell.a = np.eye(3) * 2
cell.a = [[2,0,0],[0,2,0],[0,0,2]]
cell.build()

kpts = cell.make_kpts([2,2,2])
print(kpts)

[[0.         0.         0.        ]
 [0.         0.         0.83122962]
 [0.         0.83122962 0.        ]
 [0.         0.83122962 0.83122962]
 [0.83122962 0.         0.        ]
 [0.83122962 0.         0.83122962]
 [0.83122962 0.83122962 0.        ]
 [0.83122962 0.83122962 0.83122962]]


In [3]:
mf = pyscf.pbc.scf.KRHF(cell, kpts=kpts).density_fit()
envs = mf.kernel(dump_chk=True)
mf.dump_scf_summary()


converged SCF energy = -1.04142270013402
**** SCF Summaries ****
Total Energy =                          -1.041422700134016
Nuclear Repulsion Energy =              -0.700892312698497
One-electron Energy =                    0.158742832973618
Two-electron Energy =                   -0.499273220409138


In [6]:
mf.mol.atom_coords(unit='Ang')
from pyscf.tools import molden
# molden.dump_scf(mf, "pbc.molden")

from pyscf.tools import chgcar
# pyscf.tools.chgcar.density(cell, "pbc.CHGCAR", mf.make_rdm1(), nx=60, ny=60, nz=60, resolution=None)

In [7]:
# Convert k-point sampled mean-field solution to a single gamma point supercell system
gamma_mf = pyscf.pbc.tools.k2gamma.k2gamma(mf)

In [8]:
# gamma point integrals
overlap = gamma_mf.cell.pbc_intor('int1e_ovlp')
kin = gamma_mf.cell.pbc_intor('int1e_kin')


In [9]:
# from pyscf.pbc import df
# eri = df.DF(gamma_mf.cell).get_eri()

In [10]:
# gamma_kmesh = pyscf.pbc.tools.k2gamma.kpts_to_kmesh(cell, kpts)
# gamma_cell = pyscf.pbc.tools.k2gamma.translation_vectors_for_kmesh(cell, gamma_kmesh)
# gamma_mf = pyscf.pbc.tools.k2gamma.k2gamma(mf, kmesh=gamma_kmesh)

# gamma_mf = pyscf.pbc.tools.k2gamma.k2gamma(mf)
# gamma_cell = pyscf.pbc.tools.k2gamma.
molden.dump_scf(gamma_mf, "pbc.molden")

pyscf.tools.chgcar.density(gamma_mf.cell, "pbc.CHGCAR", gamma_mf.make_rdm1(), nx=120, ny=60, nz=60, resolution=None)


In [42]:
pyscf.tools.molden.from_mo(gamma_mf.mol, "mo_can.molden", gamma_mf.mo_coeff)
C_loc = pyscf.lo.ibo.ibo(gamma_mf.mol, gamma_mf.mo_coeff)
pyscf.tools.molden.from_mo(gamma_mf.mol, "mo_loc.molden", C_loc)


WARN: ECP/PP is used. MINAO is not a good reference AO basis in IAO.

 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 6.92e-15


In [43]:
print(gamma_mf.mo_coeff.shape)
print(C_loc.shape)

(80, 80)
(80, 80)


In [57]:
a = np.argsort(-1*np.abs(np.diag(C_loc.T.dot(overlap.dot(C_loc)))))

In [129]:
C_val = C_loc[:,a[0:16]]
X = np.real(scipy.linalg.sqrtm(overlap))
Xinv = np.linalg.inv(X)
C_vir = np.eye(C_loc.shape[0]) - X.dot(C_val).dot(C_val.T).dot(X)
C_vir = Xinv.dot(C_vir)
# Sv = np.linalg.inv(np.real(scipy.linalg.sqrtm(C_vir.T.dot(overlap.dot(C_vir)))))
# C_vir = C_vir.dot(Sv)
print(C_val.shape)
print(C_vir.shape)

(80, 16)
(80, 80)


In [130]:
np.diag(C_vir.T.dot(overlap.dot(C_vir)))

array([0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.        ,
       0.45457096, 0.81848948, 0.72693956, 1.        , 1.     

In [131]:
print(np.sum(np.abs(C_val.T.dot(overlap.dot(C_val)))))
print(np.sum(np.abs(C_vir.T.dot(overlap.dot(C_vir)))))
print(np.sum(np.abs(C_vir.T.dot(overlap.dot(C_val)))))

16.00000000000107
133.7135029130356
8.419833324934036e-12
